In [31]:
import zipfile
import pathlib
import string
from collections import Counter
from typing import List

In [32]:
%load_ext autoreload

%autoreload 2

from utils import read_text, preprocess_text
from language_model import count_ngram, NGramStat
from encoder import beam_search, build_graph, decode_ngrams, encode_text, decode_text
from mcmc_decoding import mcmc_decryption

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
path_to_zip = pathlib.Path("data", "corpora.zip")

In [62]:
texts = read_text(path_to_zip, ignore_files=["WarAndPeaceEng.txt"])

In [64]:
for text in texts:
    texts[text] = preprocess_text(texts[text])

In [65]:
n_gram = 1

In [66]:
source_freq = Counter()
for name in texts:
    if name == "AnnaKarenina.txt":
        big_text = texts[name]
    else:
        source_freq.update(count_ngram(texts[name], n_gram=n_gram))

In [68]:
unigram_stat = NGramStat(source_freq)

In [69]:
vocab = set(unigram_stat._ngram_proba.keys())

# Частотное декодирование на основе частот отдельных симовлов

In [70]:
test_text = "Вронский слушал с удовольствием этот веселый лепет хорошенькой женщины, поддакивая ей, давал полушутливые советы и вообще тотчас же принял свой привычный тон обращения с этого рода женщинами. В его петербургском мире все люди разделялись на два совершенно противоположные сорта. Один низший сорт: пошлые, глупые и, главное, смешные люди, которые веруют в то, что одному мужу надо жить с одною женой, с которою он обвенчан, что девушке надо быть невинною, женщине стыдливою, мужчине мужественным, воздержным и твердым, что надо воспитывать детей, зарабатывать свой хлеб, платить долги, — и разные тому подобные глупости. Это был сорт людей старомодных и смешных. Но был другой сорт людей, настоящих, к которому они все принадлежали, в котором надо быть, главное, элегантным, красивым, великодушным, смелым, веселым, отдаваться всякой страсти не краснея и над всем остальным смеяться."

In [71]:
encoded_text, original_processed = encode_text(test_text, vocab, n_gram=1)

Encoded text length: 848


In [72]:
encoded_text

'🍨🔡🔔📘👇🚈🎣🕣🎏👇🔍🐕🆑👦🔍🎏👇🎏🐕😸🔔🍨🔔🔍📯👇⏫🍨🎣🕑😁🎏🐯⏫🔔⏫🎏🍨🕑👇🕑🔍🍷🕣🎏🔍🕑🔳🕑⏫🎏🚲🔔🔡🔔🆑🕑📘📯🚈🔔🕣🎏🚽🕑📘🍉🎣📘🍷🎏🔳🔔😸😸👦🚈🎣🍨👦❌🎏🕑🕣🎏😸👦🍨👦🔍🎏🔳🔔🔍🐕🆑🐕⏫🔍🎣🍨🍷🕑🎏👇🔔🍨🕑⏫🍷🎏🎣🎏🍨🔔🔔💣🍉🕑🎏⏫🔔⏫🍀👦👇🎏🚽🕑🎏🔳🔡🎣📘❌🔍🎏👇🍨🔔🕣🎏🔳🔡🎣🍨🍷🍀📘🍷🕣🎏⏫🔔📘🎏🔔💣🔡👦🍉🕑📘🎣❌🎏👇🎏🐯⏫🔔🐔🔔🎏🔡🔔😸👦🎏🚽🕑📘🍉🎣📘👦😁🎣🎏🍨🎏🕑🐔🔔🎏🔳🕑⏫🕑🔡💣🐕🔡🐔👇🚈🔔😁🎏😁🎣🔡🕑🎏🍨👇🕑🎏🔍🙆😸🎣🎏🔡👦🚁😸🕑🔍❌🔍🎣👇📯🎏📘👦🎏😸🍨👦🎏👇🔔🍨🕑🔡🆑🕑📘📘🔔🎏🔳🔡🔔⏫🎣🍨🔔🔳🔔🔍🔔🚽📘🍷🕑🎏👇🔔🔡⏫👦🎏🔔😸🎣📘🎏📘🎣🚁🆑🎣🕣🎏👇🔔🔡⏫🎏🔳🔔🆑🔍🍷🕑🎏🐔🔍🐕🔳🍷🕑🎏🎣🎏🐔🔍👦🍨📘🔔🕑🎏👇😁🕑🆑📘🍷🕑🎏🔍🙆😸🎣🎏🚈🔔⏫🔔🔡🍷🕑🎏🍨🕑🔡🐕🙆⏫🎏🍨🎏⏫🔔🎏🍀⏫🔔🎏🔔😸📘🔔😁🐕🎏😁🐕🚽🐕🎏📘👦😸🔔🎏🚽🎣⏫📯🎏👇🎏🔔😸📘🔔🙆🎏🚽🕑📘🔔🕣🎏👇🎏🚈🔔⏫🔔🔡🔔🙆🎏🔔📘🎏🔔💣🍨🕑📘🍀👦📘🎏🍀⏫🔔🎏😸🕑🍨🐕🆑🚈🕑🎏📘👦😸🔔🎏💣🍷⏫📯🎏📘🕑🍨🎣📘📘🔔🙆🎏🚽🕑📘🍉🎣📘🕑🎏👇⏫🍷😸🔍🎣🍨🔔🙆🎏😁🐕🚽🍀🎣📘🕑🎏😁🐕🚽🕑👇⏫🍨🕑📘📘🍷😁🎏🍨🔔🚁😸🕑🔡🚽📘🍷😁🎏🎣🎏⏫🍨🕑🔡😸🍷😁🎏🍀⏫🔔🎏📘👦😸🔔🎏🍨🔔👇🔳🎣⏫🍷🍨👦⏫📯🎏😸🕑⏫🕑🕣🎏🚁👦🔡👦💣👦⏫🍷🍨👦⏫📯🎏👇🍨🔔🕣🎏🚲🔍🕑💣🎏🔳🔍👦⏫🎣⏫📯🎏😸🔔🔍🐔🎣🎏🎣🎏🔡👦🚁📘🍷🕑🎏⏫🔔😁🐕🎏🔳🔔😸🔔💣📘🍷🕑🎏🐔🔍🐕🔳🔔👇⏫🎣🎏🐯⏫🔔🎏💣🍷🔍🎏👇🔔🔡⏫🎏🔍🙆😸🕑🕣🎏👇⏫👦🔡🔔😁🔔😸📘🍷🚲🎏🎣🎏👇😁🕑🆑📘🍷🚲🎏📘🔔🎏💣🍷🔍🎏😸🔡🐕🐔🔔🕣🎏👇🔔🔡⏫🎏🔍🙆😸🕑🕣🎏📘👦👇⏫🔔❌🍉🎣🚲🎏🚈🎏🚈🔔⏫🔔🔡🔔😁🐕🎏🔔📘🎣🎏🍨👇🕑🎏🔳🔡🎣📘👦😸🔍🕑🚽👦🔍🎣🎏🍨🎏🚈🔔⏫🔔🔡🔔😁🎏📘👦😸🔔🎏💣🍷⏫📯🎏🐔🔍👦🍨📘🔔🕑🎏🐯🔍🕑🐔👦📘⏫📘🍷😁🎏🚈🔡👦👇🎣🍨🍷😁🎏🍨🕑🔍🎣🚈🔔😸🐕🆑📘🍷😁🎏👇😁🕑🔍🍷😁🎏🍨🕑👇🕑🔍🍷😁🎏🔔⏫😸👦🍨👦⏫📯👇❌🎏🍨👇❌🚈🔔🕣🎏👇⏫🔡👦👇⏫🎣🎏📘🕑🎏🚈🔡👦👇📘🕑❌🎏🎣🎏📘👦😸🎏🍨👇🕑😁🎏🔔👇⏫👦🔍📯📘🍷😁🎏👇😁🕑❌⏫📯👇❌'

In [73]:
for serach_res in decode_text(encoded_text, unigram_stat, n_gram, beam_search_width=10):
    print(serach_res.phrase)

Beam search:   0%|          | 0/847 [00:00<?, ?it/s]Start form: (0, 'с')
сроенятг нвублв н укосовьнистам эиои санавдг вапаи юоробаеьяог ыаехтед покклятслч аг клслв повубуивтсда носаид т соозха иоишлн ыа пртечв нсог пртсдшедг иое озрлхаетч н эиойо рокл ыаехтелмт с айо паиарзурйняом мтра сна вжкт рлцкавчвтнь ел ксл носарбаеео проитсоповоыеда норил окте етцбтг нори побвда йвупда т йвлсеоа нмабеда вжкт яоиорда саружи с ио шио океому муыу елко ытиь н океож ыаеог н яоиорож ое озсаешле шио касубяа елко здиь еастееож ыаехтеа нидквтсож муыштеа муыанисаеедм соцкарыедм т исаркдм шио елко сонптидслиь каиаг цлрлзлидслиь нсог юваз пвлитиь ковйт т рлцеда иому покозеда йвупонит эио здв нори вжкаг нилромокедю т нмабедю ео здв круйог нори вжкаг елниочхтю я яоиорому оет сна пртелкваылвт с яоиором елко здиь йвлсеоа эвайлеиедм ярлнтсдм савтяокубедм нмавдм санавдм оиклслиьнч снчяог нирлнит еа ярлнеач т елк снам онилвьедм нмачиьнч


In [74]:
original_processed

'вронский слушал с удовольствием этот веселый лепет хорошенькой женщины поддакивая ей давал полушутливые советы и вообще тотчас же принял свой привычный тон обращения с этого рода женщинами в его петербургском мире все люди разделялись на два совершенно противоположные сорта один низший сорт пошлые глупые и главное смешные люди которые веруют в то что одному мужу надо жить с одною женой с которою он обвенчан что девушке надо быть невинною женщине стыдливою мужчине мужественным воздержным и твердым что надо воспитывать детей зарабатывать свой хлеб платить долги и разные тому подобные глупости это был сорт людей старомодных и смешных но был другой сорт людей настоящих к которому они все принадлежали в котором надо быть главное элегантным красивым великодушным смелым веселым отдаваться всякой страсти не краснея и над всем остальным смеяться'

# Частотное декодирование на основе частот биграмм

In [75]:
n_gram = 2
source_freq = Counter()
for name in texts:
    if name == "AnnaKarenina.txt":
        big_text = texts[name]
    else:
        source_freq.update(count_ngram(texts[name], n_gram=n_gram))

In [76]:
bigram_stat = NGramStat(source_freq)

In [77]:
encoded_text, original_processed = encode_text(test_text, vocab, n_gram=n_gram)

Encoded text length: 848


In [78]:
encoded_text

'🍨🔡🔔📘👇🚈🎣🕣🎏👇🔍🐕🆑👦🔍🎏👇🎏🐕😸🔔🍨🔔🔍📯👇⏫🍨🎣🕑😁🎏🐯⏫🔔⏫🎏🍨🕑👇🕑🔍🍷🕣🎏🔍🕑🔳🕑⏫🎏🚲🔔🔡🔔🆑🕑📘📯🚈🔔🕣🎏🚽🕑📘🍉🎣📘🍷🎏🔳🔔😸😸👦🚈🎣🍨👦❌🎏🕑🕣🎏😸👦🍨👦🔍🎏🔳🔔🔍🐕🆑🐕⏫🔍🎣🍨🍷🕑🎏👇🔔🍨🕑⏫🍷🎏🎣🎏🍨🔔🔔💣🍉🕑🎏⏫🔔⏫🍀👦👇🎏🚽🕑🎏🔳🔡🎣📘❌🔍🎏👇🍨🔔🕣🎏🔳🔡🎣🍨🍷🍀📘🍷🕣🎏⏫🔔📘🎏🔔💣🔡👦🍉🕑📘🎣❌🎏👇🎏🐯⏫🔔🐔🔔🎏🔡🔔😸👦🎏🚽🕑📘🍉🎣📘👦😁🎣🎏🍨🎏🕑🐔🔔🎏🔳🕑⏫🕑🔡💣🐕🔡🐔👇🚈🔔😁🎏😁🎣🔡🕑🎏🍨👇🕑🎏🔍🙆😸🎣🎏🔡👦🚁😸🕑🔍❌🔍🎣👇📯🎏📘👦🎏😸🍨👦🎏👇🔔🍨🕑🔡🆑🕑📘📘🔔🎏🔳🔡🔔⏫🎣🍨🔔🔳🔔🔍🔔🚽📘🍷🕑🎏👇🔔🔡⏫👦🎏🔔😸🎣📘🎏📘🎣🚁🆑🎣🕣🎏👇🔔🔡⏫🎏🔳🔔🆑🔍🍷🕑🎏🐔🔍🐕🔳🍷🕑🎏🎣🎏🐔🔍👦🍨📘🔔🕑🎏👇😁🕑🆑📘🍷🕑🎏🔍🙆😸🎣🎏🚈🔔⏫🔔🔡🍷🕑🎏🍨🕑🔡🐕🙆⏫🎏🍨🎏⏫🔔🎏🍀⏫🔔🎏🔔😸📘🔔😁🐕🎏😁🐕🚽🐕🎏📘👦😸🔔🎏🚽🎣⏫📯🎏👇🎏🔔😸📘🔔🙆🎏🚽🕑📘🔔🕣🎏👇🎏🚈🔔⏫🔔🔡🔔🙆🎏🔔📘🎏🔔💣🍨🕑📘🍀👦📘🎏🍀⏫🔔🎏😸🕑🍨🐕🆑🚈🕑🎏📘👦😸🔔🎏💣🍷⏫📯🎏📘🕑🍨🎣📘📘🔔🙆🎏🚽🕑📘🍉🎣📘🕑🎏👇⏫🍷😸🔍🎣🍨🔔🙆🎏😁🐕🚽🍀🎣📘🕑🎏😁🐕🚽🕑👇⏫🍨🕑📘📘🍷😁🎏🍨🔔🚁😸🕑🔡🚽📘🍷😁🎏🎣🎏⏫🍨🕑🔡😸🍷😁🎏🍀⏫🔔🎏📘👦😸🔔🎏🍨🔔👇🔳🎣⏫🍷🍨👦⏫📯🎏😸🕑⏫🕑🕣🎏🚁👦🔡👦💣👦⏫🍷🍨👦⏫📯🎏👇🍨🔔🕣🎏🚲🔍🕑💣🎏🔳🔍👦⏫🎣⏫📯🎏😸🔔🔍🐔🎣🎏🎣🎏🔡👦🚁📘🍷🕑🎏⏫🔔😁🐕🎏🔳🔔😸🔔💣📘🍷🕑🎏🐔🔍🐕🔳🔔👇⏫🎣🎏🐯⏫🔔🎏💣🍷🔍🎏👇🔔🔡⏫🎏🔍🙆😸🕑🕣🎏👇⏫👦🔡🔔😁🔔😸📘🍷🚲🎏🎣🎏👇😁🕑🆑📘🍷🚲🎏📘🔔🎏💣🍷🔍🎏😸🔡🐕🐔🔔🕣🎏👇🔔🔡⏫🎏🔍🙆😸🕑🕣🎏📘👦👇⏫🔔❌🍉🎣🚲🎏🚈🎏🚈🔔⏫🔔🔡🔔😁🐕🎏🔔📘🎣🎏🍨👇🕑🎏🔳🔡🎣📘👦😸🔍🕑🚽👦🔍🎣🎏🍨🎏🚈🔔⏫🔔🔡🔔😁🎏📘👦😸🔔🎏💣🍷⏫📯🎏🐔🔍👦🍨📘🔔🕑🎏🐯🔍🕑🐔👦📘⏫📘🍷😁🎏🚈🔡👦👇🎣🍨🍷😁🎏🍨🕑🔍🎣🚈🔔😸🐕🆑📘🍷😁🎏👇😁🕑🔍🍷😁🎏🍨🕑👇🕑🔍🍷😁🎏🔔⏫😸👦🍨👦⏫📯👇❌🎏🍨👇❌🚈🔔🕣🎏👇⏫🔡👦👇⏫🎣🎏📘🕑🎏🚈🔡👦👇📘🕑❌🎏🎣🎏📘👦😸🎏🍨👇🕑😁🎏🔔👇⏫👦🔍📯📘🍷😁🎏👇😁🕑❌⏫📯👇❌'

In [79]:
for serach_res in decode_text(encoded_text, bigram_stat, n_gram, beam_search_width=20):
    print()
    print(serach_res.log_likelihood, serach_res.phrase)
    print()

Beam search: 848it [00:12, 67.59it/s]               
Beam search: 4221it [00:00, 41806.26it/s]         Start form: (0, 'у')
Beam search: 16132it [00:01, 9440.79it/s] 
-825.2856673436542 уюпеётетбоъяф цуо шлимьсчеакакгоэтмуенут о и велциалф  вдогрсяблдой коанлеалылнощеан сл  санься инмоятчтмибыфешкенвальчтм умжнелчтковоя бочаё ковочукаё м анасу хоосщепеэт треы алелдой неоленситоенцужддрф ётс тоспчт пчтдикиузыеа  омору нтеантемибыовдолиены зьчаъюсев у тои отелеронзашаетбои маъюыму т ъяывчтцефиа личтвиа качтдикиеммуф у енждатматом енм елери рашиютязваревожабоелеринблдосябоеммуф  н санасбыотноннм есгля  кязвареи жа н иролих ъеотрочтнаодлочах шикнрочтшиъенабыосщиеназа ивкагоцеакжд бгопоренестенегдотьножа сцуднпуые нниеннию олся двебуатниотю стшецецее цау омс раъюстску т ъяъенацеэтреи цумицобиже нбогах  керня кболапрняязи и цемнютсябои мадие ё нена сй ь мея м ы шийстре таановнеалыш сценея м ы гонест идую фиа личтфресны нщиеме шлятгобымой елпрщибои мегобыжамегомуалножавые выя ё нае нацен еме 

In [19]:
original_processed

'вронский слушал с удовольствием этот веселый лепет хорошенькой женщины поддакивая ей давал полушутливые советы и вообще тотчас же принял свой привычный тон обращения с этого рода женщинами в его петербургском мире все люди разделялись на два совершенно противоположные сорта один низший сорт пошлые глупые и главное смешные люди которые веруют в то что одному мужу надо жить с одною женой с которою он обвенчан что девушке надо быть невинною женщине стыдливою мужчине мужественным воздержным и твердым что надо воспитывать детей зарабатывать свой хлеб платить долги и разные тому подобные глупости это был сорт людей старомодных и смешных но был другой сорт людей настоящих к которому они все принадлежали в котором надо быть главное элегантным красивым великодушным смелым веселым отдаваться всякой страсти не краснея и над всем остальным смеяться'

In [80]:
import random

In [81]:
init_vocab = sorted(vocab)
perm = init_vocab.copy()

In [82]:
random.shuffle(perm)

In [83]:
enc_mapping = {letter: perm_letter for letter, perm_letter in zip(init_vocab, perm)}

In [84]:
enc_mapping

{' ': 'м',
 'а': 'ы',
 'б': 'ж',
 'в': ' ',
 'г': 'в',
 'д': 'х',
 'е': 'о',
 'ж': 'з',
 'з': 'а',
 'и': 'ф',
 'й': 'ш',
 'к': 'е',
 'л': 'ь',
 'м': 'э',
 'н': 'щ',
 'о': 'д',
 'п': 'ё',
 'р': 'й',
 'с': 'и',
 'т': 'я',
 'у': 'ъ',
 'ф': 'ч',
 'х': 'т',
 'ц': 'у',
 'ч': 'к',
 'ш': 'ю',
 'щ': 'п',
 'ъ': 'н',
 'ы': 'б',
 'ь': 'л',
 'э': 'р',
 'ю': 'ц',
 'я': 'г',
 'ё': 'с'}

In [91]:
test_text = big_text[:1600]

In [92]:
encoded = "".join(map(lambda x: enc_mapping[x], test_text))

In [93]:
test_text

'анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива посвоему это книга о вечных ценностях о любви о вере о семье о человеческом достоинстве лев толстойроман широкого дыхания часть первая лев толстой анна каренина роман широкого дыхания анна каренина поразила современников вседневностью содержания необычайная свобода раскованность повествования удивительно сочетались в этом романе с цельностью художественного взгляда автора на жизнь он выступал здесь как художник и мыслитель и назначение искусства видел не в том чтобы неоспоримо разрешить вопрос а в том чтобы заставить любить жизнь в бесчисленных никогда не истощимых всех ее проявлениях в е годы один маститый писатель повидимому гончаров сказал достоевскому это вещь неслыханная это вещь первая кто у нас из писателей может поравняться с этим а в европе кто представит хоть чтонибудь подобное фм достоевский находил 

In [94]:
original_processed

'вронский слушал с удовольствием этот веселый лепет хорошенькой женщины поддакивая ей давал полушутливые советы и вообще тотчас же принял свой привычный тон обращения с этого рода женщинами в его петербургском мире все люди разделялись на два совершенно противоположные сорта один низший сорт пошлые глупые и главное смешные люди которые веруют в то что одному мужу надо жить с одною женой с которою он обвенчан что девушке надо быть невинною женщине стыдливою мужчине мужественным воздержным и твердым что надо воспитывать детей зарабатывать свой хлеб платить долги и разные тому подобные глупости это был сорт людей старомодных и смешных но был другой сорт людей настоящих к которому они все принадлежали в котором надо быть главное элегантным красивым великодушным смелым веселым отдаваться всякой страсти не краснея и над всем остальным смеяться'

In [95]:
encoded

'ыщщымеыйощфщымдхфщмфамиыэбтмащыэощфябтмйдэыщд мьл ымядьиядвдмщыкфщыояигмияы юошмычдйфаэдэмчйыадшм иомикыияьф бомиоэлфмёдтдзфмхйъвмщымхйъвымеызхыгмщоикыияьф ыгмиоэлгмщоикыияьф ымёди доэъмрядмещфвымдм окщбтмуощщдиягтмдмьцж фмдм ойомдмиоэломдмкоьд окоиедэмхдиядфщия омьо мядьиядшйдэыщмюфйдедвдмхбтыщфгмкыиялмёой ыгмьо мядьиядшмыщщымеыйощфщымйдэыщмюфйдедвдмхбтыщфгмыщщымеыйощфщымёдйыафьымид йоэощщфед м иохщо щдиялцмидхойзыщфгмщоджбкышщыгми дждхымйыиед ыщщдиялмёд оия д ыщфгмъхф фяоьлщдмидкояыьфилм мрядэмйдэыщомимуоьлщдиялцмтъхдзоия ощщдвдм авьгхымы ядйымщымзфащлмдщм бияъёыьмахоилмеыемтъхдзщфемфмэбиьфяоьлмфмщыащыкощфомфиеъиия ым фхоьмщом мядэмкяджбмщодиёдйфэдмйыайоюфялм дёйдимым мядэмкяджбмаыияы фялмьцжфялмзфащлм мжоикфиьощщбтмщфедвхымщомфиядпфэбтм иотмоомёйдг ьощфгтм момвдхбмдхфщмэыияфябшмёфиыяоьлмёд фхфэдэъмвдщкыйд миеыаыьмхдиядо иедэъмрядм оплмщоиьбтыщщыгмрядм оплмёой ыгмеядмъмщыимфамёфиыяоьошмэдзоямёдйы щгялигмимряфэмым мо йдёомеядмёйохияы фямтдялмкядщфжъхлмёдхджщдомчэмхдиядо иефшмщытдхфьм

In [96]:
len(encoded)

1600

In [98]:
for i in range(1):
    print(mcmc_decryption(encoded, bigram_stat, set(init_vocab), 8_000)[0])

  0%|          | 0/8000 [00:00<?, ?it/s]

анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива посвоему это книга о вечных ценностях о любви о вере о семье о человеческом достоинстве лев толстойроман широкого дыхания часть первая лев толстой анна каренина роман широкого дыхания анна каренина поразила современников вседневностью содержания необычайная свобода раскованность повествования удивительно сочетались в этом романе с цельностью художественного взгляда автора на жизнь он выступал здесь как художник и мыслитель и назначение искусства видел не в том чтобы неоспоримо разрешить вопрос а в том чтобы заставить любить жизнь в бесчисленных никогда не истощимых всех ее проявлениях в е годы один маститый писатель повидимому гончаров сказал достоевскому это вещь неслыханная это вещь первая кто у нас из писателей может поравняться с этим а в европе кто представит хоть чтонибудь подобное фм достоевский находил в